In [41]:
import pandas as pd
import re

# Load the provided CSV files
fonologia_df = pd.read_excel('fonologia transposat_complet_MDPN.xlsx')
world_atlas_df = pd.read_csv('World-Atlas-of-Language-Structures.csv')

fonologia_df_transposed = fonologia_df.T
fonologia_df_transposed.columns = fonologia_df_transposed.iloc[0]
fonologia_df_transposed = fonologia_df_transposed[1:]

fonologia_df_transposed['Language'] = fonologia_df_transposed.index
fonologia_df_transposed.reset_index(drop=True, inplace=True)

cols = list(fonologia_df_transposed.columns)
cols = [cols[-1]] + cols[:-1]
fonologia_df_transposed = fonologia_df_transposed[cols]


fonologia_df_transposed['Countries'] = 'Unknown'
fonologia_df_transposed['Macroarea'] = 'Unknown'
fonologia_df_transposed['Geoposition'] = 'Unknown'

# Iterate over the rows of the fonologia dataframe

for index, row in fonologia_df_transposed.iterrows():

    # Get the name of the language
    language_name = row['Language']

    # Find the row in the world atlas dataframe that corresponds to the current language
    atlas_row = world_atlas_df[world_atlas_df['Name'] == language_name]

    # If the language is found in the world atlas dataframe
    if len(atlas_row) > 0:
        # Get the country, macroarea, and geoposition of the language
        pattern = r'Entity\["Country", "(.*?)"\]'
        country = re.findall(pattern, atlas_row['Countries'].values[0])
        macroarea = atlas_row['Macroarea'].values[0]
        pattern = r'GeoPosition\[{([^{}]+)}\]'
        geoposition = re.findall(pattern, atlas_row['GeoPosition'].values[0])

        # Update the corresponding row in the fonologia dataframe
        fonologia_df_transposed.at[index, 'Countries'] = country
        fonologia_df_transposed.at[index, 'Macroarea'] = macroarea
        fonologia_df_transposed.at[index, 'Geoposition'] = geoposition


# Save the updated fonologia dataframe to a new CSV file
fonologia_df_transposed.to_csv('fonologia_complete.csv', index=False)
fonologia_df_transposed.to_excel('fonologia_complete.xlsx', index=False)





In [15]:
# Read the geoposition information from fonologia_df and paint a point in a map for each location, all locations in the same map

import folium
from folium import plugins
import pandas as pd

fonologia_df = pd.read_csv('fonologia_complete.csv')

# Create a map centered at the equator
m = folium.Map(location=[0, 0], zoom_start=2)

# Iterate over the rows of the fonologia dataframe

for index, row in fonologia_df.iterrows():
    
        # Get the geoposition of the language
        geoposition = row['Geoposition']
    
        # If the geoposition is not "Unknown"
        if geoposition != 'Unknown':
            # Split the geoposition into latitude and longitude
            latitude, longitude = geoposition[1:-1].split(',')
    
            # Add a marker to the map at the latitude and longitude
            folium.Marker([float(latitude[1:]), float(longitude[:-1])], popup=row['Language']).add_to(m)

# Plot the map

m.save('language_location_map.html')

